# 4.

In [2]:
import gensim
import pandas as pd
import tensorflow as tf
import numpy as np
from tensorflow.keras import layers
from tensorflow.keras import losses
from tensorflow.keras import preprocessing
from tensorflow.keras.layers.experimental.preprocessing import TextVectorization
from keras.models import Sequential
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder

C:\Users\diego\anaconda3\envs\TF-Keras\lib\site-packages\gensim\similarities\__init__.py:15: UserWarning: The gensim.similarities.levenshtein submodule is disabled, because the optional Levenshtein package <https://pypi.org/project/python-Levenshtein/> is unavailable. Install Levenhstein (e.g. `pip install python-Levenshtein`) to suppress this warning.
  warnings.warn(msg)


Readind the raw data and enconding the main characters

In [68]:
ruta = './docs/'

In [3]:
'''
Aquí se filtra el dataset para que solo queden los dialogos relevantes, los de ls personajes principales.
'''
# Dataset from - https://archive.ics.uci.edu/ml/datasets/SMS+Spam+Collection
df_0 = pd.read_csv(ruta+'simpsons_dataset.csv').dropna()
target_simpsons = [
    'Lisa Simpson',
    'Bart Simpson',
    'Homer Simpson',
    'Marge Simpson',
    'Maggie Simpson',
    'Grampa Simpson'
]
df = df_0.loc[df_0['raw_character_text'].isin(target_simpsons)]


'''
Se realiza un Encoding de los nombres de los personajes principales.
La idea es que cada categoría de personaje este codificada por un vector
1x6 binario.
'''
enc = OneHotEncoder(handle_unknown='ignore')
enc.fit(df[['raw_character_text']])
print(enc.categories_)


[array(['Bart Simpson', 'Grampa Simpson', 'Homer Simpson', 'Lisa Simpson',
       'Maggie Simpson', 'Marge Simpson'], dtype=object)]


Dataset Spliting

In [4]:
'''
Se dividen los datos en 60 training, 20 validación y 20 testing
'''
# df_yelp = df[df['source'] == 'yelp']
sentences = df['spoken_words'].values
y = enc.transform(df[['raw_character_text']]).toarray()
sentences_train_val, sentences_test, y_train_val, y_test = train_test_split(sentences, y, test_size=0.20, random_state=1000)
sentences_train, sentences_val, y_train, y_val = train_test_split(sentences_train_val, y_train_val, test_size=0.25, random_state=1000)

# format: a tuple of the form (text_data, label).
train_dataset = tf.data.Dataset.from_tensor_slices((sentences_train, y_train))
test_dataset = tf.data.Dataset.from_tensor_slices((sentences_test, y_test))
val_dataset = tf.data.Dataset.from_tensor_slices((sentences_val, y_val))

# We also create a dataset with only the textual data in it. This will be used to build our vocabulary later on.
text_dataset = tf.data.Dataset.from_tensor_slices((sentences_train))

Building the vocabulary

In [5]:
vectorize_layer = TextVectorization(
    ngrams=None, max_tokens=None, vocabulary=None,
    output_mode='binary', output_sequence_length=None, pad_to_max_tokens=True,
)

vectorize_layer.adapt(text_dataset.batch(32))
print(len(vectorize_layer.get_vocabulary()))


20742


In [43]:

x_train = vectorize_layer(np.array([[s] for s in sentences_train])).numpy()
x_val = vectorize_layer(np.array([[s] for s in sentences_val])).numpy()
x_test = vectorize_layer(np.array([[s] for s in sentences_test])).numpy()

y_train = np.array(y_train)
y_val = np.array(y_val)
y_test = np.array(y_test)

Define model genereting function

In [6]:
def generateInplaceEmbbedingModel(embedding_dim, vectorize_layer, output_length = len(target_simpsons), architecture_id = 1):
    batch_size = 32
    # epochs = 10
    num_tokens = len(vectorize_layer.get_vocabulary())
    embedding_layer = layers.Embedding(
        num_tokens,
        embedding_dim,
        name="embedding_Trainable"
    )
    model = Sequential()
    model.add(vectorize_layer)
    model.add(embedding_layer)
    model.add(layers.GlobalAveragePooling1D())

    if architecture_id == 1:
        model.add(layers.Dense(10, activation='relu'))
    elif architecture_id == 2:
        model.add(layers.Dense(80, activation='relu'))
        model.add(layers.Dense(250, activation='relu'))
    elif architecture_id == 3:
        model.add(layers.Dense(80, activation='relu'))
        model.add(layers.Dropout(0.3))
        model.add(layers.BatchNormalization())
        model.add(layers.Dense(250, activation='relu'))
        model.add(layers.Dropout(0.5))

    model.add(layers.Dense(output_length, activation='softmax'))

    return model

Definition of vectorization layer generator

In [7]:
def getVectorizeLayer(output_mode, text_dataset):
    batch_size = 32
    vectorize_layer = TextVectorization(
        ngrams=None,
        max_tokens=None, vocabulary=None,
        output_mode=output_mode, output_sequence_length=None, pad_to_max_tokens=True,
    )
    vectorize_layer.adapt(text_dataset.batch(batch_size))
    print( f'Output mode: {output_mode}', 'Vocab length:', len(vectorize_layer.get_vocabulary()))
    return vectorize_layer

Get All 3 vectorize layers

In [8]:
vectorize_bin = getVectorizeLayer('binary', text_dataset)
vectorize_int = getVectorizeLayer('int', text_dataset)
vectorize_tfidf = getVectorizeLayer('tf-idf', text_dataset)

Output mode: binary Vocab length: 20742
Output mode: int Vocab length: 20743
Output mode: tf-idf Vocab length: 20742


Definition of constants

In [9]:
embedding_dim = 100
epochs = 10
batch_size = 32

#### Architecture 1

In [10]:
model1_bin = generateInplaceEmbbedingModel(embedding_dim, vectorize_bin, architecture_id = 1)
model1_int = generateInplaceEmbbedingModel(embedding_dim, vectorize_int, architecture_id = 1)
model1_tfidf = generateInplaceEmbbedingModel(embedding_dim, vectorize_tfidf, architecture_id = 1)

Compile models

In [11]:
model1_bin.compile(loss='categorical_crossentropy',  optimizer='adam', metrics=['accuracy'])
model1_int.compile(loss='categorical_crossentropy',  optimizer='adam', metrics=['accuracy'])
model1_tfidf.compile(loss='categorical_crossentropy',  optimizer='adam', metrics=['accuracy'])

Architecture 1 Summary

Training

In [12]:
model1_bin.fit(train_dataset.batch(batch_size), validation_data=test_dataset.batch(batch_size), epochs=epochs)

Epoch 1/10
1249/1249 [==============================] - 183s 146ms/step - loss: 1.6667 - accuracy: 0.4153 - val_loss: 1.4757 - val_accuracy: 0.4155
Epoch 2/10
1249/1249 [==============================] - 189s 151ms/step - loss: 1.4540 - accuracy: 0.4186 - val_loss: 1.4287 - val_accuracy: 0.4155
Epoch 3/10
1249/1249 [==============================] - 181s 145ms/step - loss: 1.4190 - accuracy: 0.4186 - val_loss: 1.4158 - val_accuracy: 0.4155
Epoch 4/10
1249/1249 [==============================] - 178s 143ms/step - loss: 1.4082 - accuracy: 0.4186 - val_loss: 1.4101 - val_accuracy: 0.4155
Epoch 5/10
1249/1249 [==============================] - 179s 143ms/step - loss: 1.4032 - accuracy: 0.4186 - val_loss: 1.4070 - val_accuracy: 0.4155
Epoch 6/10
1249/1249 [==============================] - 181s 145ms/step - loss: 1.4005 - accuracy: 0.4186 - val_loss: 1.4053 - val_accuracy: 0.4155
Epoch 7/10
1249/1249 [==============================] - 185s 148ms/step - loss: 1.3989 - accuracy: 0.4186 - val_

In [69]:
model1_int.fit(train_dataset.batch(batch_size), validation_data=test_dataset.batch(batch_size), epochs=epochs)

Epoch 1/10
1249/1249 [==============================] - 25s 20ms/step - loss: 1.1221 - accuracy: 0.5436 - val_loss: 1.2578 - val_accuracy: 0.4845
Epoch 2/10
1249/1249 [==============================] - 25s 20ms/step - loss: 1.0397 - accuracy: 0.5920 - val_loss: 1.2754 - val_accuracy: 0.4865
Epoch 3/10
1249/1249 [==============================] - 26s 21ms/step - loss: 0.9722 - accuracy: 0.6267 - val_loss: 1.3077 - val_accuracy: 0.4814
Epoch 4/10
1249/1249 [==============================] - 25s 20ms/step - loss: 0.9133 - accuracy: 0.6516 - val_loss: 1.3438 - val_accuracy: 0.4785
Epoch 5/10
1249/1249 [==============================] - 25s 20ms/step - loss: 0.8586 - accuracy: 0.6721 - val_loss: 1.3866 - val_accuracy: 0.4733
Epoch 6/10
1249/1249 [==============================] - 25s 20ms/step - loss: 0.8110 - accuracy: 0.6888 - val_loss: 1.4323 - val_accuracy: 0.4721
Epoch 7/10
1249/1249 [==============================] - 25s 20ms/step - loss: 0.7705 - accuracy: 0.7025 - val_loss: 1.4803 -

In [14]:
model1_tfidf.fit(train_dataset.batch(batch_size), validation_data=test_dataset.batch(batch_size), epochs=epochs)

Epoch 1/10
1249/1249 [==============================] - 195s 156ms/step - loss: 1.4875 - accuracy: 0.4130 - val_loss: 1.4041 - val_accuracy: 0.4155
Epoch 2/10
1249/1249 [==============================] - 193s 155ms/step - loss: 1.3982 - accuracy: 0.4186 - val_loss: 1.4036 - val_accuracy: 0.4155
Epoch 3/10
1249/1249 [==============================] - 197s 158ms/step - loss: 1.3980 - accuracy: 0.4186 - val_loss: 1.4035 - val_accuracy: 0.4155
Epoch 4/10
1249/1249 [==============================] - 207s 166ms/step - loss: 1.3979 - accuracy: 0.4186 - val_loss: 1.4034 - val_accuracy: 0.4155
Epoch 5/10
1249/1249 [==============================] - 198s 158ms/step - loss: 1.3978 - accuracy: 0.4186 - val_loss: 1.4034 - val_accuracy: 0.4155
Epoch 6/10
1249/1249 [==============================] - 199s 159ms/step - loss: 1.3978 - accuracy: 0.4186 - val_loss: 1.4033 - val_accuracy: 0.4155
Epoch 7/10
1249/1249 [==============================] - 187s 150ms/step - loss: 1.3977 - accuracy: 0.4186 - val_

In [15]:
model1_bin.summary()
model1_int.summary()
model1_tfidf.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
text_vectorization_1 (TextVe (None, 20742)             0         
_________________________________________________________________
embedding_Trainable (Embeddi (None, 20742, 100)        2074200   
_________________________________________________________________
global_average_pooling1d (Gl (None, 100)               0         
_________________________________________________________________
dense (Dense)                (None, 10)                1010      
_________________________________________________________________
dense_1 (Dense)              (None, 6)                 66        
Total params: 2,075,276
Trainable params: 2,075,276
Non-trainable params: 0
_________________________________________________________________
Model: "sequential_1"
_________________________________________________________________
Layer (type)            

In [26]:
# model1_bin.save(ruta)
# model1_int.save(ruta+ 'simp_model1_int')
# model1_tfidf.save( ruta+'simp_model1_tfidf')

# del model1_bin
# del model1_int
# del model1_tfidf

INFO:tensorflow:Assets written to: ./docs/assets


Architecture 2

In [27]:
model2_bin = generateInplaceEmbbedingModel(embedding_dim, vectorize_bin, architecture_id = 2)
model2_int = generateInplaceEmbbedingModel(embedding_dim, vectorize_int, architecture_id = 2)
model2_tfidf = generateInplaceEmbbedingModel(embedding_dim, vectorize_tfidf, architecture_id = 2)

Compile models

In [29]:
model2_bin.compile(loss='categorical_crossentropy',  optimizer='adam', metrics=['accuracy'])
model2_int.compile(loss='categorical_crossentropy',  optimizer='adam', metrics=['accuracy'])
model2_tfidf.compile(loss='categorical_crossentropy',  optimizer='adam', metrics=['accuracy'])

Train models

In [30]:
model2_bin.fit(train_dataset.batch(batch_size), validation_data=test_dataset.batch(batch_size), epochs=5)

Epoch 1/5
1249/1249 [==============================] - 190s 152ms/step - loss: 1.4305 - accuracy: 0.4134 - val_loss: 1.4076 - val_accuracy: 0.4155
Epoch 2/5
1249/1249 [==============================] - 185s 148ms/step - loss: 1.4000 - accuracy: 0.4186 - val_loss: 1.4059 - val_accuracy: 0.4155
Epoch 3/5
1249/1249 [==============================] - 190s 152ms/step - loss: 1.3993 - accuracy: 0.4186 - val_loss: 1.4050 - val_accuracy: 0.4155
Epoch 4/5
1249/1249 [==============================] - 194s 155ms/step - loss: 1.3988 - accuracy: 0.4186 - val_loss: 1.4043 - val_accuracy: 0.4155
Epoch 5/5
1249/1249 [==============================] - 193s 155ms/step - loss: 1.3985 - accuracy: 0.4186 - val_loss: 1.4039 - val_accuracy: 0.4155


In [64]:
model2_int.fit(train_dataset.batch(batch_size), validation_data=test_dataset.batch(batch_size), epochs=10)

Epoch 1/10
1249/1249 [==============================] - 26s 21ms/step - loss: 0.7054 - accuracy: 0.7266 - val_loss: 1.6815 - val_accuracy: 0.4685
Epoch 2/10
1249/1249 [==============================] - 25s 20ms/step - loss: 0.6542 - accuracy: 0.7470 - val_loss: 1.8669 - val_accuracy: 0.4610
Epoch 3/10
1249/1249 [==============================] - 26s 21ms/step - loss: 0.6058 - accuracy: 0.7646 - val_loss: 2.1383 - val_accuracy: 0.4549
Epoch 4/10
1249/1249 [==============================] - 26s 21ms/step - loss: 0.5633 - accuracy: 0.7777 - val_loss: 2.4012 - val_accuracy: 0.4441
Epoch 5/10
1249/1249 [==============================] - 25s 20ms/step - loss: 0.5269 - accuracy: 0.7930 - val_loss: 2.6530 - val_accuracy: 0.4432
Epoch 6/10
1249/1249 [==============================] - 26s 21ms/step - loss: 0.4929 - accuracy: 0.8066 - val_loss: 2.9147 - val_accuracy: 0.4255
Epoch 7/10
1249/1249 [==============================] - 26s 21ms/step - loss: 0.4594 - accuracy: 0.8181 - val_loss: 3.2666 -

In [32]:
model2_tfidf.fit(train_dataset.batch(batch_size), validation_data=test_dataset.batch(batch_size), epochs=5)

Epoch 1/5
1249/1249 [==============================] - 188s 150ms/step - loss: 1.4317 - accuracy: 0.4156 - val_loss: 1.4071 - val_accuracy: 0.4155
Epoch 2/5
1249/1249 [==============================] - 193s 155ms/step - loss: 1.4000 - accuracy: 0.4186 - val_loss: 1.4056 - val_accuracy: 0.4155
Epoch 3/5
1249/1249 [==============================] - 181s 145ms/step - loss: 1.3993 - accuracy: 0.4186 - val_loss: 1.4048 - val_accuracy: 0.4155
Epoch 4/5
1249/1249 [==============================] - 177s 142ms/step - loss: 1.3988 - accuracy: 0.4186 - val_loss: 1.4043 - val_accuracy: 0.4155
Epoch 5/5
1249/1249 [==============================] - 179s 143ms/step - loss: 1.3984 - accuracy: 0.4186 - val_loss: 1.4039 - val_accuracy: 0.4155


In [ ]:
# saveModel(model2_bin, 'simp_model2_bin')
# saveModel(model2_int, 'simp_model2_int')
# saveModel(model2_tfidf, 'simp_model2_tfidf')

Summary

In [33]:
model2_bin.summary()
model2_int.summary()
model2_tfidf.summary()

Model: "sequential_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
text_vectorization_1 (TextVe (None, 20742)             0         
_________________________________________________________________
embedding_Trainable (Embeddi (None, 20742, 100)        2074200   
_________________________________________________________________
global_average_pooling1d_3 ( (None, 100)               0         
_________________________________________________________________
dense_6 (Dense)              (None, 80)                8080      
_________________________________________________________________
dense_7 (Dense)              (None, 250)               20250     
_________________________________________________________________
dense_8 (Dense)              (None, 6)                 1506      
Total params: 2,104,036
Trainable params: 2,104,036
Non-trainable params: 0
____________________________________________

Architecture 3


In [34]:
model3_bin = generateInplaceEmbbedingModel(embedding_dim, vectorize_bin, architecture_id = 3)
model3_int = generateInplaceEmbbedingModel(embedding_dim, vectorize_int, architecture_id = 3)
model3_tfidf = generateInplaceEmbbedingModel(embedding_dim, vectorize_tfidf, architecture_id = 3)

Compile models

In [35]:
model3_bin.compile(loss='categorical_crossentropy',  optimizer='adam', metrics=['accuracy'])
model3_int.compile(loss='categorical_crossentropy',  optimizer='adam', metrics=['accuracy'])
model3_tfidf.compile(loss='categorical_crossentropy',  optimizer='adam', metrics=['accuracy'])

Train models

In [36]:
model3_bin.fit(train_dataset.batch(batch_size), validation_data=test_dataset.batch(batch_size), epochs=epochs)

Epoch 1/10
1249/1249 [==============================] - 183s 146ms/step - loss: 1.4483 - accuracy: 0.3996 - val_loss: 1.4049 - val_accuracy: 0.4155
Epoch 2/10
1249/1249 [==============================] - 183s 147ms/step - loss: 1.4029 - accuracy: 0.4186 - val_loss: 1.4041 - val_accuracy: 0.4155
Epoch 3/10
1249/1249 [==============================] - 182s 146ms/step - loss: 1.4005 - accuracy: 0.4186 - val_loss: 1.4038 - val_accuracy: 0.4155
Epoch 4/10
1249/1249 [==============================] - 182s 146ms/step - loss: 1.3997 - accuracy: 0.4186 - val_loss: 1.4034 - val_accuracy: 0.4155
Epoch 5/10
1249/1249 [==============================] - 182s 146ms/step - loss: 1.3994 - accuracy: 0.4186 - val_loss: 1.4034 - val_accuracy: 0.4155
Epoch 6/10
1249/1249 [==============================] - 184s 147ms/step - loss: 1.3994 - accuracy: 0.4186 - val_loss: 1.4034 - val_accuracy: 0.4155
Epoch 7/10
1249/1249 [==============================] - 183s 147ms/step - loss: 1.3987 - accuracy: 0.4186 - val_

In [65]:
model3_int.fit(train_dataset.batch(batch_size), validation_data=test_dataset.batch(batch_size), epochs=10)

Epoch 1/10
1249/1249 [==============================] - 28s 22ms/step - loss: 0.4967 - accuracy: 0.8056 - val_loss: 3.2164 - val_accuracy: 0.4143
Epoch 2/10
1249/1249 [==============================] - 27s 22ms/step - loss: 0.4789 - accuracy: 0.8127 - val_loss: 3.2977 - val_accuracy: 0.4352
Epoch 3/10
1249/1249 [==============================] - 28s 22ms/step - loss: 0.4702 - accuracy: 0.8158 - val_loss: 3.1451 - val_accuracy: 0.4143
Epoch 4/10
1249/1249 [==============================] - 27s 22ms/step - loss: 0.4583 - accuracy: 0.8182 - val_loss: 3.6846 - val_accuracy: 0.4549
Epoch 5/10
1249/1249 [==============================] - 28s 22ms/step - loss: 0.4566 - accuracy: 0.8204 - val_loss: 3.4131 - val_accuracy: 0.4472
Epoch 6/10
1249/1249 [==============================] - 27s 22ms/step - loss: 0.4441 - accuracy: 0.8254 - val_loss: 3.5354 - val_accuracy: 0.4471
Epoch 7/10
1249/1249 [==============================] - 28s 22ms/step - loss: 0.4380 - accuracy: 0.8269 - val_loss: 3.5897 -

In [39]:
model3_tfidf.fit(train_dataset.batch(batch_size), validation_data=test_dataset.batch(batch_size), epochs=5)

Epoch 1/5
1249/1249 [==============================] - 178s 142ms/step - loss: 1.4492 - accuracy: 0.3989 - val_loss: 1.4055 - val_accuracy: 0.4155
Epoch 2/5
1249/1249 [==============================] - 180s 144ms/step - loss: 1.4016 - accuracy: 0.4186 - val_loss: 1.4047 - val_accuracy: 0.4155
Epoch 3/5
1249/1249 [==============================] - 184s 148ms/step - loss: 1.4009 - accuracy: 0.4186 - val_loss: 1.4037 - val_accuracy: 0.4155
Epoch 4/5
1249/1249 [==============================] - 179s 143ms/step - loss: 1.4009 - accuracy: 0.4186 - val_loss: 1.4036 - val_accuracy: 0.4155
Epoch 5/5
1249/1249 [==============================] - 180s 144ms/step - loss: 1.4002 - accuracy: 0.4186 - val_loss: 1.4035 - val_accuracy: 0.4155


Summary

In [40]:
model3_bin.summary()
model3_int.summary()
model3_tfidf.summary()

Model: "sequential_6"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
text_vectorization_1 (TextVe (None, 20742)             0         
_________________________________________________________________
embedding_Trainable (Embeddi (None, 20742, 100)        2074200   
_________________________________________________________________
global_average_pooling1d_6 ( (None, 100)               0         
_________________________________________________________________
dense_15 (Dense)             (None, 80)                8080      
_________________________________________________________________
dropout (Dropout)            (None, 80)                0         
_________________________________________________________________
batch_normalization (BatchNo (None, 80)                320       
_________________________________________________________________
dense_16 (Dense)             (None, 250)              

In [ ]:
# saveModel(model3_bin, 'simp_model3_bin')
# saveModel(model3_int, 'simp_model3_int')
# saveModel(model3_tfidf, 'simp_model3_tfidf')

Evaluate

In [60]:
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
import warnings
warnings.filterwarnings("ignore")


def get_metrics_by_class(model, x,y):
  y_pred = model.predict(x.batch(64), verbose=0)
  y_pred_bool = np.argmax(y_pred, axis=1)
  y_label = np.argmax(y, axis=1)
  print(classification_report(y_label, y_pred_bool))

Evaluate 1st Architecture

In [70]:
print('Architecture 1 binary Embedding')
get_metrics_by_class(model1_bin, test_dataset, y_test)
print('Architecture 1 int Embedding')
get_metrics_by_class(model1_int, test_dataset, y_test)
print('Architecture 1 tfidf Embedding')
get_metrics_by_class(model1_tfidf, test_dataset, y_test)

Architecture 1 binary Embedding
              precision    recall  f1-score   support

           0       0.00      0.00      0.00      2621
           1       0.00      0.00      0.00       381
           2       0.42      1.00      0.59      5533
           3       0.00      0.00      0.00      2145
           4       0.00      0.00      0.00         3
           5       0.00      0.00      0.00      2635

    accuracy                           0.42     13318
   macro avg       0.07      0.17      0.10     13318
weighted avg       0.17      0.42      0.24     13318

Architecture 1 int Embedding
              precision    recall  f1-score   support

           0       0.41      0.27      0.32      2621
           1       0.29      0.08      0.13       381
           2       0.54      0.64      0.59      5533
           3       0.36      0.30      0.33      2145
           4       0.00      0.00      0.00         3
           5       0.39      0.47      0.43      2635

    accuracy    

Evaluate Second Architecture

In [67]:
print('Architecture 2 binary Embedding')
get_metrics_by_class(model2_bin, test_dataset, y_test)
print('Architecture 2 int Embedding')
get_metrics_by_class(model2_int, test_dataset, y_test)
print('Architecture 2 tfidf Embedding')
get_metrics_by_class(model2_tfidf, test_dataset, y_test)

Architecture 2 binary Embedding
              precision    recall  f1-score   support

           0       0.00      0.00      0.00      2621
           1       0.00      0.00      0.00       381
           2       0.42      1.00      0.59      5533
           3       0.00      0.00      0.00      2145
           4       0.00      0.00      0.00         3
           5       0.00      0.00      0.00      2635

    accuracy                           0.42     13318
   macro avg       0.07      0.17      0.10     13318
weighted avg       0.17      0.42      0.24     13318

Architecture 2 int Embedding
              precision    recall  f1-score   support

           0       0.36      0.22      0.28      2621
           1       0.23      0.11      0.15       381
           2       0.55      0.56      0.55      5533
           3       0.31      0.38      0.34      2145
           4       0.00      0.00      0.00         3
           5       0.38      0.46      0.41      2635

    accuracy    

In [66]:
print('Architecture 3 binary Embedding')
get_metrics_by_class(model3_bin, test_dataset, y_test)
print('Architecture 3 int Embedding')
get_metrics_by_class(model3_int, test_dataset, y_test)
print('Architecture 3 tfidf Embedding')
get_metrics_by_class(model3_tfidf, test_dataset, y_test)

Architecture 3 binary Embedding
              precision    recall  f1-score   support

           0       0.00      0.00      0.00      2621
           1       0.00      0.00      0.00       381
           2       0.42      1.00      0.59      5533
           3       0.00      0.00      0.00      2145
           4       0.00      0.00      0.00         3
           5       0.00      0.00      0.00      2635

    accuracy                           0.42     13318
   macro avg       0.07      0.17      0.10     13318
weighted avg       0.17      0.42      0.24     13318

Architecture 3 int Embedding
              precision    recall  f1-score   support

           0       0.33      0.38      0.35      2621
           1       0.12      0.17      0.14       381
           2       0.50      0.70      0.58      5533
           3       0.37      0.14      0.21      2145
           4       0.00      0.00      0.00         3
           5       0.51      0.20      0.29      2635

    accuracy    